# MCMC3.5: Jackknife Resampling

To estimate the error of observables accurately, it is recommended to use the Jackknife resampling method.

## Binning

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3
"""
Metropolis method
"""
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(rand())
end
"""
Generating a honeycomb lattice with an open boundary condition.
"""
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten((zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1)))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end
@resumable function measurementflux(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

One reason of the error bar is the discreteness of the returned value `flux`.

In [2]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 25.0, 4, 4), 10000)
foreach(println, Iterators.take(mcstep, 10))

0.3333333333333333
0.16666666666666666
0.3333333333333333
-0.16666666666666666
0.0
0.3333333333333333
0.0
0.3333333333333333
0.6666666666666666
0.6666666666666666


We have to flatten these quantized values by binning.

In [3]:
using Statistics
Nsample = 10000
Nbin = 10
Nbinsize = Nsample ÷ Nbin
iter = Iterators.partition(Iterators.take(mcstep, Nsample), Nbinsize)
bin = collect(map(mean, iter))

10-element Array{Float64,1}:
 0.30949999999999994
 0.3198333333333333 
 0.31649999999999995
 0.30650000000000005
 0.29700000000000004
 0.311              
 0.3136666666666667 
 0.3031666666666667 
 0.29683333333333334
 0.2868333333333333 

Now it works!

In [4]:
m = mean(bin)
s = stdm(bin, m) / sqrt(length(bin))
println("$m ± $s")

0.3060833333333333 ± 0.0032287529603567287


`Nbinsize` has to be determined based on the autocorrelation. In order to reduce `Nbin` to get a more acculate result at low temperature, we need to implement some global updating algorithm.

## Delete-1 jackknife resampling

Delete-1 jackknife resampling is simply implemented in https://github.com/ararslan/Jackknife.jl. However, the function is limited, so I will newly define functions for the jackknife resampling.

In [5]:
"""
Delete one sample from given samples. before and after are applied before and after taking an expectation value.
"""
function leaveoneout(before::Function, after::Function, v::AbstractVector)
    ind = eachindex(v)
    map(i -> after(mean(map(before, view(v, filter(!isequal(i), ind))))), ind)
end
"""
Calculate an expectation value by jackknife.
"""
meanJ(b::Function, a::Function, v::AbstractVector) = mean(leaveoneout(b, a, v))
"""
Calculate an error bar with given mean by jackknife.
"""
stdmJ(b::Function, a::Function, v::AbstractVector, m) = stdm(leaveoneout(b, a, v), m, corrected = false) * sqrt(length(v) - 1)
"""
Calculate an error bar by jackknife.
"""
stdJ(b::Function, a::Function, v::AbstractVector) = stdmJ(b, a, v, meanJ(b, a, v))

stdJ

The functions are based on Statistics.jl and Jackknife.jl, so please see their reference to know how it works. I again use `measurementEf` as a demonstration.

In [6]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    β₂ = β * 0.5
    hdense = Array(h)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] ^ 2) * (sech(β₂ * ev[(N >> 1 + 1) : end]) ^ 2)) * 0.25
        @yield [Ef, ∂Ef∂β]
    end
end

measurementEf (generic function with 1 method)

It is ok to first assume the bin size to be 1.

In [7]:
const β = 1.0
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, 4, 4), 10000)
iter2 = Iterators.take(mcstep2, Nsample)
data = collect(iter2)

10000-element Array{Array{Float64,1},1}:
 [-4.20902, -3.10331]
 [-4.21159, -3.1137] 
 [-4.21045, -3.10931]
 [-4.21146, -3.11295]
 [-4.20787, -3.09891]
 [-4.20798, -3.09948]
 [-4.20656, -3.09348]
 [-4.20799, -3.09957]
 [-4.209, -3.10314]  
 [-4.20938, -3.10529]
 [-4.20659, -3.09371]
 [-4.21146, -3.11295]
 [-4.21275, -3.11818]
 ⋮                   
 [-4.20912, -3.10384]
 [-4.20914, -3.10392]
 [-4.21394, -3.12285]
 [-4.21043, -3.10922]
 [-4.21266, -3.11772]
 [-4.20556, -3.09003]
 [-4.20658, -3.09362]
 [-4.21043, -3.10922]
 [-4.20903, -3.10334]
 [-4.21405, -3.12342]
 [-4.20917, -3.10416]
 [-4.20928, -3.10481]

By setting `after` = `before` = `identity`, `meanJ` and `stdmJ `work in the same way as `mean` and `stdm`, respectively.

In [8]:
m2 = meanJ(identity, identity, data)
s2 = stdmJ(identity, identity, data, m2)
println("Ef = $(m2[1]) ± $(s2[1]), ∂Ef∂β = $(m2[2]) ± $(s2[2])")

Ef = -4.209183350941615 ± 2.107301929003753e-5, ∂Ef∂β = -3.1041772230109714 ± 8.283878730766585e-5


This agrees with the standard estimation method for the error bars.

In [9]:
std(data) / sqrt(length(data))

2-element Array{Float64,1}:
 2.107301929068348e-5
 8.283878730819529e-5

For such mean values, the jackknife resampling is apparently overkill. However, to estimate the error for the values like the specific heat, the jackknife resampling is very effective.

In [10]:
"""
Function applied before mean to calculate heat capacity.
"""
TTCv(v::Vector{Float64}) = [v[1] ^ 2 - v[2], v[1]]
"""
Function applied after mean to calculate heat capacity.
"""
function Cv(β::Float64)::Function
    β² = β ^ 2
    meanTTCv::Vector{Float64} -> β² * (meanTTCv[1] - meanTTCv[2] ^ 2)
end
m3 = meanJ(TTCv, Cv(β), data)
s3 = stdmJ(TTCv, Cv(β), data, m3)
println("Cv = $m3 ± $s3")

Cv = 3.104181663288264 ± 8.283942216920118e-5


## Autocorrelation

The simplest way to estimate autocorrelation is by changing the size of binning and estimating its errors by jackknife resampling.

In [11]:
binning = Iterators.partition(data, 2)
bin2 = mean.(binning)

5000-element Array{Array{Float64,1},1}:
 [-4.21031, -3.1085] 
 [-4.21095, -3.11113]
 [-4.20793, -3.0992] 
 [-4.20727, -3.09653]
 [-4.20919, -3.10421]
 [-4.20903, -3.10333]
 [-4.21036, -3.1088] 
 [-4.21089, -3.11074]
 [-4.20851, -3.10144]
 [-4.20844, -3.10102]
 [-4.20916, -3.10414]
 [-4.21278, -3.11839]
 [-4.21031, -3.10851]
 ⋮                   
 [-4.20966, -3.10588]
 [-4.2092, -3.10432] 
 [-4.20855, -3.10168]
 [-4.21031, -3.10854]
 [-4.20844, -3.10108]
 [-4.21102, -3.1115] 
 [-4.20913, -3.10388]
 [-4.21219, -3.11603]
 [-4.20911, -3.10387]
 [-4.2085, -3.10142] 
 [-4.21154, -3.11338]
 [-4.20923, -3.10448]

In [12]:
m4 = meanJ(TTCv, Cv(β), bin2)
s4 = stdmJ(TTCv, Cv(β), bin2, m4)
println("Cv = $m4 ± $s4")

Cv = 3.1041794974051182 ± 8.385019353534018e-5


The fact that the binsize does not affect the expectation value (or the errorbar) too much means that the autocorrelation length is about 1 step. Note that at low temperature the binsize strongly affects the expectation value, which means that the binsize must be taken to be large enough. Here I estimate the autocorrelation length by another method.

In [13]:
using StatsBase
StatsBase.autocor(first.(data))

41-element Array{Float64,1}:
  1.0                  
  0.012202974151887293 
  0.009776170543310888 
  0.007150727590217465 
  0.000395589294995162 
 -0.016826661519978664 
  0.009420548587773327 
  0.015804130580611378 
 -0.004622283251176312 
  0.0025919554738979118
 -0.015079510610355954 
 -0.0010452719974011185
  0.0024925013897314483
  ⋮                    
 -0.0020773736792928975
 -0.0021790609156399974
 -0.01138202236218162  
  0.010260051851941515 
  0.0021036163103581593
  0.017425518572686753 
  0.0002287512925417109
 -0.014409977905247135 
 -0.0004435549787670428
  0.00305473918804639  
 -0.0008334122687307837
  0.001242981033175764 

Rapid decay in the autocorrelation function means that the autocorrelation length is less than 1.

## Bootstrap method

~ under construction ~

**Exercise**: implement a Bootstrap method.